# Clase que implementa un Algoritmo evolutivo

Cada individuo se forma por un cromosoma formado por los siguientes genes:

**Hiperparámetros estructurales**:

- *loopback_window*: 
    rango:  0,7 
    bits:   3
    coding: 2^i
    values: 0=1, 1=2, 2=4, 3=8, 4=16, 5=32, 6=64, 7=128
- *forward_window*:
    rango:  0,7
    bits:   3
    coding: 1+i
    values: 0=1, 1=2, 2=3, 3=4, 4=5, 5=6, 6=7, 7=8
- *num_lstm_layers*:
    rango:  0,3
    bits:   2
    coding: 1+i
    values: 0=1, 1=2, 2=3, 3=4
- *num_dense_layers*: 
    rango:  0,1
    bits:   1
    coding: 2+i
    values: 0=1, 1=2
- *num_cells_per_layer*:
    rango:  (0,3)
    bits:   4
    coding: default: 32, bit0=64 bit1=128 bit2=256 bit3=512 
    values: 0=32, 1=64, 2=128, 3=192, ..., 15=960=512+256+128+64

**Hiperparámetros de entrenamiento**:

- *batch_size*: 
    rango:  (0,3)
    bits:   4
    coding: default: 1, bit0=8 bit1=16 bit2=32 bit3=64 
    values: 0=1, 1=8, 2=16, 3=24, ..., 15=120=64+32+16+8
- *suffling_enable*: rango (0,1), se codifica con 1 bits: 0=>(False), 1=>(True)
    rango:  (0,1)
    bits:   1
    coding: (i) 
    values: 0=False, 1=True

En resumen, cada individuo consta de 3+3+2+1+4+4+1 = 18 bits

In [2]:
####################################################################################
# Chequeo si está operativo el entorno de Google Colaboratory
import sys
ENABLE_GOOGLE_COLAB = 'google.colab' in sys.modules
ENABLE_GOOGLE_COLAB

False

In [3]:
# cargo módulos necesarios para trabajar en google colab
if ENABLE_GOOGLE_COLAB:
    !pip install deap
    !pip install bitstring
    !pip install PyDrive
    !wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
    !tar -xzvf ta-lib-0.4.0-src.tar.gz
    %cd ta-lib
    !./configure --prefix=/usr
    !make
    !make install
    !pip install Ta-Lib
    import talib


In [4]:
if ENABLE_GOOGLE_COLAB:
    from google.colab import files
    src = list(files.upload().values())[0]
    open('PredictiveNet.py','wb').write(src)

In [5]:
if ENABLE_GOOGLE_COLAB:
    src = list(files.upload().values())[0]
    open('EURUSD_H1.csv','wb').write(src)


In [6]:
from PredictiveNet import PredictiveNet
from deap import base, creator, tools, algorithms
from scipy.stats import bernoulli
from bitstring import BitArray
import numpy as np
import pickle
import sys
import traceback

#######################################################################
#######################################################################
#######################################################################

#-------------------------------------------------------------
def predictive_net_decoder(chromosome):
  """
  Decodifica un chromosoma en una solución tipo 'ind = list(features)'. En este caso contamos con 7 features:
  lbw : loopback-window, 3-bits
  fww : forward-window, 2-bits
  nll : num-lstm-layers, 1-bits
  ndl : num-dense-layers, 1-bit
  nlc : num-layer-cells, 2-bits
  bs  : batch-size, 3-bits
  sf  : suffle-flag, 1-bit   

  Returns:
    Individuo, como una lista de features ind = [lbw,fww,nll,ndl,nlc,bs,sf], que será utilizado para realizar la evaluación
  """        
  chromo =[BitArray(chromosome[0:3]).uint, BitArray(chromosome[3:5]).uint, BitArray(chromosome[5:6]).uint,
           BitArray(chromosome[6:7]).uint, BitArray(chromosome[7:9]).uint, BitArray(chromosome[9:12]).uint,
           BitArray(chromosome[12:13]).uint]
  ind = []
  # lbw
  lbw = [4, 8, 12, 24, 48, 72, 96, 168]
  ind.append(lbw[chromo[0]])
  # fww
  fww = [1,2,4,8]
  ind.append(fww[chromo[1]])
  # nll
  nll = [1,2]
  ind.append(nll[chromo[2]])
  # ndl
  ndl = [1,2]
  ind.append(ndl[chromo[3]])
  # nlc
  nlc = [64, 128, 256, 512]
  ind.append(nlc[chromo[4]])  
  # bs
  bs = [1, 4, 8, 16, 32, 64, 128, 256]
  ind.append(bs[chromo[5]])  
  # sf
  sf = [False, True]
  ind.append(sf[chromo[6]]) 
  return ind     


#-------------------------------------------------------------
def predictive_net_evaluator(ind):
  """
  Evalúa un individuo, devolviendo su fitness (0,1).
  Args:
    ind : Individual

  Returns:
    fitness del individuo en el rango 0,1
  """       
  nn = PredictiveNet(ind.name,
                     loopback_window=ind.genes[0], 
                     forward_window=ind.genes[1], 
                     num_lstm_layers=ind.genes[2], 
                     num_dense_layers=ind.genes[3],
                     num_cells=ind.genes[4], 
                     batch_size=ind.genes[5],
                     suffle_enable= ind.genes[6],
                     tvt_csv_file='EURUSD_H1.csv')   
  epochs = 2
  for i in range(epochs):
    history = nn.train_validate(epochs)
    loss = history.history['val_loss']
    acc = history.history['val_acc']
    if acc == 0:
      print('Ind={}, loss={}, acc={}'.format(ind.name,loss,acc))      
      return acc      
  loss,acc = nn.test_eval()
  print('Ind={}, loss={}, acc={}'.format(ind.name,loss,acc))      
  return acc


#-------------------------------------------------------------
def simplistic_evaluator(ind):
  """
  Evalúa un individuo, devolviendo su fitness (0,1).
  Args:
    ind : Individual

  Returns:
    fitness del individuo en el rango 0,1
  """       
  acc = (ind.genes[2] + ind.genes[4])/1026
  print('Ind={}, acc={}'.format(ind.name,acc))      
  return acc


#-------------------------------------------------------------
def save_to_file(obj, filepath):
  with open(filepath, 'wb') as f:
    try:
      pickle.dump(obj, f)
    except MemoryError as error:
      exc_type, exc_value, exc_traceback = sys.exc_info()
      print('type:', exc_type, 'value:', exc_value)
        
        
#-------------------------------------------------------------
def load_from_file(filepath):
  with open(filepath, 'rb') as f:
    obj = pickle.load(f)        
  return obj   

#######################################################################
#######################################################################
#######################################################################

class Individual:
  def __init__(self, chromosome, decoder):
    """
      Crea un un individuo a partir de un cromosoma binario
      Args:
        chromosome: Cromosoma binario generado por deap.toolbox
        decoder   : Callback de decodificación del cromosoma
    """
    self.genes = decoder(chromosome)
    self.name = str(self.genes)                 

        
#######################################################################
#######################################################################
#######################################################################

class GA:
  def __init__(self, popsize, genesize, chr_decoder, ind_evaluator):
    """
      Crea el algoritmo genético
      Args:
        popsize       : Tamaño de la población
        genesize      : Número de genes en el cromosoma
        chr_decoder   : Callback para decodificar un cromosoma en un individuo evaluable
        ind_evaluator : Callback para evaluar individuos
    """      
    self.toolbox = None
    self.popsize = popsize
    self.genesize = genesize
    self.chr_decoder = chr_decoder
    self.ind_evaluator = ind_evaluator
    self.restart()

    # As we are trying to minimize the RMSE score, that's why using -1.0. 
    # In case, when you want to maximize accuracy for instance, use 1.0
    print('Creando GA...')
    creator.create('FitnessMax', base.Fitness, weights = (1.0,))
    creator.create('Individual', list , fitness = creator.FitnessMax)
    toolbox = base.Toolbox()
    toolbox.register('binary', bernoulli.rvs, 0.5)
    toolbox.register('individual', tools.initRepeat, creator.Individual, toolbox.binary, n = self.genesize)
    toolbox.register('population', tools.initRepeat, list , toolbox.individual)
    toolbox.register('mate', tools.cxOrdered)
    toolbox.register('mutate', tools.mutFlipBit, indpb = 0.05)
    toolbox.register('select', tools.selRoulette)
    toolbox.register('evaluate', self.evaluate_individual)
    self.population = toolbox.population(n = self.popsize)
    self.toolbox = toolbox        
    self.summary()
                                 
  #-------------------------------------------------------------
  def summary(self):
    print('Individuals:')
    for chromo in self.population:
      print(chromo)
                                 
  #-------------------------------------------------------------
  def evaluate_individual(self, chromosome):
    ind = Individual(chromosome, self.chr_decoder)
    fitness = self.ind_evaluator(ind)
    return [fitness]    
                        
  #-------------------------------------------------------------
  def restart(self):
    self.generations = 0
    self.results = []
    
  #-------------------------------------------------------------
  def execute(self, numgens, pcross, pmut):
    print('Ejecutando GA de generación={} a generación={}'.format(self.generations, self.generations + numgens - 1))
    self.stats = tools.Statistics(lambda ind: ind.fitness.values)
    self.stats.register("avg", np.mean)
    self.stats.register("std", np.std)
    self.stats.register("min", np.min)
    self.stats.register("max", np.max)    
    self.hallOfFame = tools.HallOfFame(self.popsize/2)
    self.results = []
    for g in range(numgens):
      print('-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-')            
      pop,log = algorithms.eaSimple(self.population, self.toolbox, cxpb = pcross, mutpb = pmut, ngen = 1, stats = self.stats, halloffame=self.hallOfFame, verbose = False)
      self.generations += 1
      results.append({'gen': self.generations,'fitmax':log[1]['max'], 'fitavg': log[1]['avg'], 'fitstd': log[1]['std'], 'fitmin': log[1]['min'], 'best': self.hallOfFame[0]})
      print(results[-1])
      obj = {'pop': self.population, 'results': self.results}
      filename = 'ga_'+str(self.generations)+'.pickle'
      save_to_file(obj, filename)
      files.download(filename)
      
    return results
    
        
#######################################################################
#######################################################################
#######################################################################




Using TensorFlow backend.


In [7]:
popsize = 10
genesize = 13
ga = GA(popsize, genesize, predictive_net_decoder, predictive_net_evaluator)

Creando GA...
Individuals:
[0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0]
[0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0]
[0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0]
[1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0]
[1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1]
[0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1]
[1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1]
[0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1]
[1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0]


In [ ]:
num_generations = 5
pcross = 0.5
pmut = 0.01
results = ga.execute(num_generations, pcross, pmut)

Ejecutando GA de generación=0 a generación=4
-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-


In [ ]:
results